reference: https://www.geeksforgeeks.org/coroutine-in-python/?ref=gcse <br>
# Coroutine in Python

함수는 명령어들의 서열이다. 그리고 subroutine들은 main함수에 의해서 호출된다.
그리고 subroutine은 단일 진입점(entry point)을 갖는다.

Coroutines are generalizations of subroutines.
- Unlike subroutines, coroutines have many entry points for suspending and resuming execution. Coroutine can suspend its execution and transfer control to other coroutine and can resume again execution from the point it left off.
- Unlike subroutines, there is no main function to call coroutines in a particular order and coordinate the results. Coroutines are cooperative that means they link together to form a pipeline. One coroutine may consume input data and send it to other that process it. Finally, there may be a coroutine to display the result.
![](./coroutine.png)

Coroutine VS Threads <br>
둘의 차이는 threads는 OS 혹은 환경이 scheduler를 통해서 전환한다면 <br>
coroutine은 프로그래머 혹은 언어가 전환할 시점을 정한다. <br>


coroutine은 pipeline을 만들기 위해 사용될 수 있다. <br>
source(producer)는 주로 coroutine은 아니고 간단한 함수지만 전체 파이프라인을 시작한다. <br>
sink는 endpoint로 모든 데이터를 모으거나 표시한다. <br>
![nn](./pipeline.png)
![nn](./pipeline2.png)

In [9]:
# coroutine chaining

def producer(sentence, next_coroutine):
    '''
    producer는 단지 문자열을 쪼개서
    pattern_filter에 하나씩 먹인다.
    '''
    tokens = sentence.split(" ")
    for token in tokens:
        next_coroutine.send(token)
    next_coroutine.close()
    # 다음 코루틴이 GeneratorExit 예외를 발생시키면 도미노처럼 닫힌다.

def pattern_filter(pattern="ing", next_coroutine=None):
    '''
    수신한 토큰에서 패턴을 찾고 패턴과 일치하는 것이 있으면
    print_token() coroutine으로 송신한다.
    '''

    print(f'Searching for {pattern}')
    try:
        while True:
            token = yield
            
            if pattern in token:
                next_coroutine.send(token)
    except GeneratorExit:
        print("Done with filtering!")
        next_coroutine.close()
        # 명확하게 뒷부분을 끝내기 위해 추가로 내가 작성해 넣었다.
        

def print_token():
    '''
    sink로 작용하며 받은 토큰들을 출력한다.
    '''
    print('this is a sink and prints tokens')
    try:
        while True:
            token = (yield) # tuple unpacking 때문에 () 쓰든 말든 결과가 똑같다. 
            print(token)
    except GeneratorExit:
        print("Done with printing!")
    


In [10]:
pt = print_token()
next(pt)
pf = pattern_filter(next_coroutine = pt)
next(pf)

sentence = "Bob is running behind a fast moving car"
producer(sentence, pf)

this is a sink and prints tokens
Searching for ing
running
moving
Done with filtering!
Done with printing!
